# DLISIO for oDa 2020

## Importing

In [ ]:
import dlisio
dlisio.set_encodings(['latin1'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly_express as px
import hvplot
import hvplot.pandas
import holoviews as hv
import panel as pn

##Let's load the plotting extensions we need for Jupyter

hv.extension('bokeh')
pn.extension()

##And let's make our Jupyter cells a bit bigger and easier to look at

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 60%; }
</style>
"""))

%matplotlib inline

## We will use a file representing signals acquired in the subsurface inside of a well from the open source NOPIMS Australia Dataset (https://nopims.dmp.wa.gov.au/Nopims/)

In [ ]:
filepath = r"PE010408.dlis"

In [ ]:
file = dlisio.load(filepath)

In [ ]:
file.describe()

## What is the origin metadata?

In [ ]:
for d in file:
    for origin in d.origins:
        print(origin.describe())

## Let's get the actual well log data now...

In [ ]:
for d in file:
    frame_count = 0
    for frame in d.frames:
        if frame_count == 0:
            curves = frame.curves() ## Magic is here
            df = pd.DataFrame(curves, index=curves[frame.index])
            frame_count = frame_count + 1

## Well log data liberated from a 1982 file format...

In [ ]:
df

## Visualize

In [ ]:
curve_list = list(df.columns)
curve_list.remove('DEPT')
curve_list.remove('FRAMENO')

In [ ]:
def curve_plot(log, df, depthname):
  aplot = df.hvplot(x=depthname, y=log, invert=True, flip_yaxis=True, shared_axes=True,
                       height=600, width=300).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14})
  return aplot;

plotlist = [curve_plot(x, df=df, depthname='DEPT') for x in curve_list]
well_section = hv.Layout(plotlist).cols(len(curve_list))
well_section